In [42]:
import numpy as np
import json
import requests
from sentence_transformers import SentenceTransformer

In [43]:
# Load the summary file
with open('Summary 2 (general overview)/output.json', 'r', encoding='utf-8') as f:
    api_output_json = json.load(f)

# Get the current summary  
current_summary = api_output_json['summary']

block_summaries = api_output_json['summaries']

source_blocks = api_output_json['text']

In [44]:
print(block_summaries)

[{'title': 'Introduction to Game Theory Concepts', 'overview': "This text introduces Game Theory as the formal study of strategic interaction where agents' outcomes are interdependent. It covers the historical development, key elements of games, different types of strategies (pure, mixed, correlated), concepts like dominated strategies, iterated deletion, rationalizability, and Nash equilibrium, illustrating them with various examples and discussing underlying assumptions like common knowledge and rationality.", 'summary': '# 🎲 **Introduction to Game Theory**\n\n- Game Theory is the formal study of **strategic interaction**.\n- In strategic settings, outcomes depend on the actions of multiple agents.\n- Predicting opponents\' play and responding optimally is key.\n- Game Theory applies to diverse areas: poker, chess, economics, business, politics, biology, computer science, etc.\n- Modern economic research heavily incorporates Game Theory, with multiple Nobel laureates in the field.\n\

In [45]:
def groq_ask(prompt, model="llama3-70b-8192"):
    # Load your API key from file
    with open("groq_api_key.txt", "r") as f:
        api_key = f.read().strip()

    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

#     # Build the full system prompt
#     system_prompt = """
# You are an assistant helping to process user edit requests about a Final Summary (FS).

# The FS was generated by summarizing several smaller Block Summaries (BS[i]), each corresponding to a part of the original Source Text (SRC[i]).

# The task is to classify each user request into one of five labels.  
# We want to minimize the amount of context we load (FS, BS[i], SRC[i]) to save tokens and speed up processing.  
# Prefer the simplest context needed to fulfill the user's request.

# Here are the possible labels:

# 1. FS_ONLY
#    • Use only the Final Summary (FS).
#    • Example user requests: "Summarize even shorter", "Make it simpler", "Translate", "Format with bullets".

# 2. FS_PLUS_BS[i]
#    • Use FS plus one best-matching Block Summary (BS[i]).
#    • Example user requests: "Explain Nash equilibrium better", "Expand the section on prisoners' dilemma", "Give a better intro for Beauty Contest".
#    • (You will later find the best block using retrieval.)

# 3. FS_PLUS_BS_ALL
#    • Use FS plus all Block Summaries.
#    • Example user requests: "Add more technical details throughout", "Include examples from earlier sections", "Enrich all topics".

# 4. FS_PLUS_SRC[i]
#    • Use FS plus a small snippet of Source Text (SRC) around a specific highlight or concept.
#    • Example user requests: "Quote the exact formula from the slide", "Insert precise numbers into the table", "Cite direct text from page 5".

# 5. FS_PLUS_SRC
#    • Use FS plus large parts of the Source Text (SRC).
#    • Example user requests: "Rewrite the summary fully but keep every mathematical proof", "Expand with original citations", "Include detailed derivations from full text".

# RULES:
# - If the user highlighted text and asks to "remove" or "delete" → assume FS_ONLY.
# - If unsure, pick the class that needs less context (e.g., FS_ONLY before FS_PLUS_BS[i], FS_PLUS_BS[i] before FS_PLUS_SRC[i], etc.).
# - Output exactly one label, no explanations, no markdown, no extra text.
# """

#     # Insert FS and User Request into the context
#     combined_prompt = f"""{system_prompt}

# ---
# FINAL SUMMARY:
# {summary}

# ---
# USER REQUEST:
# {user_prompt}

# ---
# YOUR TASK:
# Decide on exactly one label from: [FS_ONLY, FS_PLUS_BS[i], FS_PLUS_BS_ALL, FS_PLUS_SRC[i], FS_PLUS_SRC]

# OUTPUT FORMAT:
# Write only the label. No explanation. No markdown. No extra text. Only the label.

# EXAMPLE OUTPUTS:
# FS_ONLY
# FS_PLUS_BS[i]
# FS_PLUS_BS_ALL
# FS_PLUS_SRC[i]
# FS_PLUS_SRC

# NOW, YOUR ANSWER:"""

    # Build the payload
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    # Send the request
    response = requests.post(url, headers=headers, json=data)

    # Error handling
    if response.status_code != 200:
        raise Exception(f"Groq API error {response.status_code}: {response.text}")

    result = response.json()
    return result["choices"][0]["message"]["content"].strip()


In [46]:
def build_classifier_prompt(user_prompt, summary):
    # Build the full system prompt
    system_prompt = """
    You are an assistant helping to process user edit requests about a Final Summary (FS).

    The FS was generated by summarizing several smaller Block Summaries (BS[i]), each corresponding to a part of the original Source Text (SRC[i]).

    The task is to classify each user request into one of five labels.  
    We want to minimize the amount of context we load (FS, BS[i], SRC[i]) to save tokens and speed up processing.  
    Prefer the simplest context needed to fulfill the user's request.

    Here are the possible labels:

    1. FS_ONLY
    • Use only the Final Summary (FS).
    • Example user requests: "Summarize even shorter", "Make it simpler", "Translate", "Format with bullets".

    2. FS_PLUS_BS[i]
    • Use FS plus one best-matching Block Summary (BS[i]).
    • Example user requests: "Explain Nash equilibrium better", "Expand the section on prisoners' dilemma", "Give a better intro for Beauty Contest".
    • (You will later find the best block using retrieval.)

    3. FS_PLUS_BS_ALL
    • Use FS plus all Block Summaries.
    • Example user requests: "Add more technical details throughout", "Include examples from earlier sections", "Enrich all topics".

    4. FS_PLUS_SRC[i]
    • Use FS plus a small snippet of Source Text (SRC) around a specific highlight or concept.
    • Example user requests: "Quote the exact formula from the slide", "Insert precise numbers into the table", "Cite direct text from page 5".

    5. FS_PLUS_SRC
    • Use FS plus large parts of the Source Text (SRC).
    • Example user requests: "Rewrite the summary fully but keep every mathematical proof", "Expand with original citations", "Include detailed derivations from full text".

    RULES:
    - If the user highlighted text and asks to "remove" or "delete" → assume FS_ONLY.
    - If unsure, pick the class that needs less context (e.g., FS_ONLY before FS_PLUS_BS[i], FS_PLUS_BS[i] before FS_PLUS_SRC[i], etc.).
    - Output exactly one label, no explanations, no markdown, no extra text.
    """

    # Insert FS and User Request into the context
    combined_prompt = f"""

    ---
    FINAL SUMMARY:
    {summary}

    ---
    {system_prompt}
    ---
    USER REQUEST:
    {user_prompt}

    ---
    YOUR TASK:
    Decide on exactly one label from: [FS_ONLY, FS_PLUS_BS[i], FS_PLUS_BS_ALL, FS_PLUS_SRC[i], FS_PLUS_SRC]

    OUTPUT FORMAT:
    Write only the label. No explanation. No markdown. No extra text. Only the label.

    EXAMPLE OUTPUTS:
    FS_ONLY
    FS_PLUS_BS[i]
    FS_PLUS_BS_ALL
    FS_PLUS_SRC[i]
    FS_PLUS_SRC

    NOW, YOUR ANSWER:"""

    return combined_prompt

In [47]:

def openrouter_ask(combined_prompt, model="meta-llama/llama-3.3-70b-instruct"):
    # Load your API key from file
    with open("openrouter_api_key.txt", "r") as f:
        api_key = f.read().strip()

    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Build the payload
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": combined_prompt}
        ],
        "temperature": 0.0,
    }

    # Send the request
    response = requests.post(url, headers=headers, json=data)

    # Error handling
    if response.status_code != 200:
        raise Exception(f"OpenRouter API error {response.status_code}: {response.text}")
    result = response.json()
    # print("DEBUG response:", result)

    # if "choices" not in result:
    #     raise Exception(f"'choices' key missing in API response: {result}")

    return result["choices"][0]["message"]["content"].strip()

In [48]:
import time

# --- Define Test Cases ---
test_cases = [
    # FS_ONLY
    ("Summarize the key points of Nash equilibrium more briefly.", "FS_ONLY"),
    ("Rewrite the section about dominated strategies in a simpler way.", "FS_ONLY"),
    ("Can you list the main assumptions behind Game Theory in bullet points?", "FS_ONLY"),

    # FS_PLUS_BS[i]
    ("Expand the explanation of the Beauty Contest and why it matters.", "FS_PLUS_BS[i]"),
    ("Explain better how Cournot's model led to early Game Theory development.", "FS_PLUS_BS[i]"),
    ("Clarify the concept of risk dominance in the Stag Hunt example.", "FS_PLUS_BS[i]"),

    # FS_PLUS_BS_ALL
    ("Add more technical examples across the text, like Rock-Paper-Scissors and Penalty Kicks.", "FS_PLUS_BS_ALL"),
    ("Include more details and mathematical structure across all examples, not just in the introduction.", "FS_PLUS_BS_ALL"),
    ("Enrich the summary by incorporating discussions about fixed-point theorems and their applications.", "FS_PLUS_BS_ALL"),

    # FS_PLUS_SRC[i]
    ("Insert the exact payoff matrix from the Advertising War between Coke and Pepsi.", "FS_PLUS_SRC[i]"),
    ("Quote the specific definition of Nash Equilibrium from the original text.", "FS_PLUS_SRC[i]"),
    ("Add the formula for expected utility used in mixed strategies.", "FS_PLUS_SRC[i]"),

    # FS_PLUS_SRC
    ("Rewrite the entire summary but include full proofs from Kakutani's Fixed-Point Theorem.", "FS_PLUS_SRC"),
    ("Expand the summary using full original derivations related to Rationalizability and Cournot Duopoly.", "FS_PLUS_SRC"),
    ("Include complete citations and direct text from the real examples discussed in Game Theory.", "FS_PLUS_SRC"),
]

# --- Run Evaluation ---
correct = 0
total = len(test_cases)

print("Evaluating classifier...\n")

labeled_prompts = []

for i, (user_prompt, expected_label) in enumerate(test_cases, start=1):
    try:
        classifier_prompt = build_classifier_prompt(user_prompt, current_summary)
        prediction = openrouter_ask(classifier_prompt)
        prediction = prediction.strip()
        
        labeled_prompts.append({
            "prompt": user_prompt,
            "predicted_class": prediction
        })

        match = (prediction == expected_label)
        correct += int(match)

        print(f"Test {i}:")
        print(f"User Prompt: {user_prompt}")
        print(f"Expected: {expected_label}")
        print(f"Predicted: {prediction}")
        print(f"{'✅ Correct' if match else '❌ Incorrect'}")
        print("-" * 50)

        # time.sleep(5)  # Slight delay to not spam the API
    except Exception as e:
        print(f"Test {i}: ❌ Error - {str(e)}")
        print("-" * 50)

# --- Final Report ---
accuracy = (correct / total) * 100
print(f"\n✅ Final Accuracy: {correct}/{total} ({accuracy:.2f}%)")

Evaluating classifier...

Test 1:
User Prompt: Summarize the key points of Nash equilibrium more briefly.
Expected: FS_ONLY
Predicted: FS_ONLY
✅ Correct
--------------------------------------------------
Test 2:
User Prompt: Rewrite the section about dominated strategies in a simpler way.
Expected: FS_ONLY
Predicted: FS_PLUS_BS[i]
❌ Incorrect
--------------------------------------------------
Test 3:
User Prompt: Can you list the main assumptions behind Game Theory in bullet points?
Expected: FS_ONLY
Predicted: FS_ONLY
✅ Correct
--------------------------------------------------
Test 4:
User Prompt: Expand the explanation of the Beauty Contest and why it matters.
Expected: FS_PLUS_BS[i]
Predicted: FS_PLUS_BS[i]
✅ Correct
--------------------------------------------------
Test 5:
User Prompt: Explain better how Cournot's model led to early Game Theory development.
Expected: FS_PLUS_BS[i]
Predicted: FS_PLUS_BS[i]
✅ Correct
--------------------------------------------------
Test 6:
User P

In [49]:
scoped_request_test_cases = [
    # (highlighted_text, user_prompt, expected_class)

    # Clarification Requests
    ("Game Theory is the formal study of strategic interaction.", "Explain what 'strategic interaction' means.", "FS_ONLY"),
    ("Predicting opponents' play and responding optimally is key.", "Can you explain how players predict opponents' play?", "FS_PLUS_BS[i]"),
    ("The field boomed from the 1980s onwards, with recent applications in computer science, political science, psychology, and evolutionary biology.", "Expand on the recent applications mentioned here.", "FS_PLUS_BS[i]"),

    # Simplification Requests
    ("The game structure (N, S, u) is typically common knowledge.", "Simplify this sentence for a beginner.", "FS_ONLY"),
    ("A strategy si is strictly dominated by σi' if σi' yields a strictly higher payoff than si for player i, regardless of the opponents' strategies.", "Can you rephrase this in simpler words?", "FS_PLUS_SRC[i]"),

    # Style Change Requests
    ("A Nash equilibrium is a strategy profile with the property that no player can benefit by deviating from his corresponding strategy.", "Rewrite this to sound more academic.", "FS_PLUS_BS[i]"),

    # Expansion Requests
    ("Rock-Paper-Scissors: Illustrates simple payoff structures.", "Add an example of how Rock-Paper-Scissors payoff works.", "FS_ONLY"),
    ("Penalty kicks: An example of a real-world strategic interaction often analyzed using game theory concepts like mixed strategies and simultaneous moves.", "Explain why penalty kicks are a good example of mixed strategies.", "FS_ONLY"),
]


# --- Run Evaluation ---
correct = 0
total = len(scoped_request_test_cases)

print("Evaluating classifier with highlighted text...\n")

scoped_labeled_prompts = []

for i, (highlighted_text, user_prompt, expected_label) in enumerate(scoped_request_test_cases, start=1):
    try:
        classifier_prompt = build_classifier_prompt(highlighted_text, user_prompt)
        prediction = openrouter_ask(classifier_prompt)
        prediction = prediction.strip()
        
        scoped_labeled_prompts.append({
            "highlighted_text": highlighted_text,
            "user_prompt": user_prompt,
            "predicted_class": prediction
        })

        match = (prediction == expected_label)
        correct += int(match)

        print(f"Test {i}:")
        print(f"Highlighted Text: {highlighted_text}")
        print(f"User Request: {user_prompt}")
        print(f"Expected: {expected_label}")
        print(f"Predicted: {prediction}")
        print(f"{'✅ Correct' if match else '❌ Incorrect'}")
        print("-" * 50)

        # time.sleep(5)  # Optional API delay
    except Exception as e:
        print(f"Test {i}: ❌ Error - {str(e)}")
        print("-" * 50)

# --- Final Report ---
accuracy = (correct / total) * 100
print(f"\n✅ Final Accuracy: {correct}/{total} ({accuracy:.2f}%)")

Evaluating classifier with highlighted text...

Test 1:
Highlighted Text: Game Theory is the formal study of strategic interaction.
User Request: Explain what 'strategic interaction' means.
Expected: FS_ONLY
Predicted: FS_ONLY
✅ Correct
--------------------------------------------------
Test 2:
Highlighted Text: Predicting opponents' play and responding optimally is key.
User Request: Can you explain how players predict opponents' play?
Expected: FS_PLUS_BS[i]
Predicted: FS_ONLY
❌ Incorrect
--------------------------------------------------
Test 3:
Highlighted Text: The field boomed from the 1980s onwards, with recent applications in computer science, political science, psychology, and evolutionary biology.
User Request: Expand on the recent applications mentioned here.
Expected: FS_PLUS_BS[i]
Predicted: FS_PLUS_BS[i]
✅ Correct
--------------------------------------------------
Test 4:
Highlighted Text: The game structure (N, S, u) is typically common knowledge.
User Request: Simplify 

In [50]:
def build_prompt_fs_only(final_summary: str, user_prompt: str, highlighted_text: str = None) -> str:
    if highlighted_text is None:

        return f"""You are given a Final Summary (FS) generated from a full text.

The user wants to edit the Final Summary without needing any additional source material.  
Use only the FS provided. Do not use any external knowledge or fabricate any content.

Perform the user's edit request as accurately as possible, working only with the FS.

---

FINAL SUMMARY:
{final_summary}

---
USER REQUEST:
{user_prompt}

---
Respond by directly editing the Final Summary as requested.
"""
    else:
        # Highlighted text provided: Focused edit
        return f"""You are given a Final Summary (FS) generated from a full text.

The user highlighted a specific portion of the Final Summary, enclosed in [brackets], and requested an edit for that part.  
Focus **only** on the highlighted text inside [brackets] when responding.  
Use only the FS provided. Do not use any external knowledge or fabricate any content.

---

FINAL SUMMARY:
{final_summary}

---
HIGHLIGHTED TEXT:
[{highlighted_text}]

---
USER REQUEST:
{user_prompt}

---
Respond by editing or expanding only the highlighted portion inside [brackets] based on the user's request.  
Keep the edit consistent with the overall style and tone of the Final Summary.
"""


In [51]:
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_with_local_model(text):
    # Accepts a string or a list of strings
    if isinstance(text, str):
        text = [text]
    
    embeddings = embedding_model.encode(text, normalize_embeddings=True)
    return embeddings

In [52]:
def inject_embeddings(summaries):
    for item in summaries:
        item["embedding"] = embed_with_local_model(item["overview"])
    return summaries

block_summaries = inject_embeddings(block_summaries)

In [53]:
def find_best_matching_index(summaries, prompt):
    prompt_embedding = embed_with_local_model(prompt)
    overview_embeddings = [item["embedding"] for item in summaries]
    
    # Compute cosine similarities in one line
    similarities = [np.dot(emb.squeeze(), prompt_embedding.squeeze()) for emb in overview_embeddings]
    
    # Find best matching index
    best_index = np.argmax(similarities)
    return best_index

In [54]:
def build_prompt_fs_plus_bs(
    block_summary: str,
    final_summary: str,
    user_prompt: str,
    highlighted_text: str = None
) -> str:
    """
    Builds the prompt when the task requires FS + a related Block Summary (BS),
    optionally focusing on a highlighted section of the Final Summary.
    """

    if highlighted_text is None:
        # General request
        return f"""You are given two pieces of information:
- Final Summary (FS): A complete summary of the entire text.
- Block Summary (BS): A summary of a specific section of the text, provided as additional context.

The user's request concerns the Final Summary or the full text.  
Use the FS as the primary source of information.  
Refer to the BS only if it helps clarify or expand on information in the FS.  
Do not use any external knowledge or fabricate any content.

Answer the user's request as accurately as possible, prioritizing information from the FS, and using the BS only as supporting context if needed.

---

FINAL SUMMARY (FS):
{final_summary}

---
BLOCK SUMMARY (BS):
{block_summary}

---
USER REQUEST:
{user_prompt}

---
Respond based mainly on the Final Summary.  
Use the Block Summary only for additional clarification if necessary.  
Do not invent new facts or introduce information not present in the FS or BS.
"""
    else:
        # Scoped request (highlighted portion of the FS)
        return f"""You are given two pieces of information:
- Final Summary (FS): A complete summary of the entire text.
- Block Summary (BS): A summary of a specific section of the text, provided as additional context.

The user highlighted a specific portion of the Final Summary, enclosed in [brackets], and requested an edit for that part.  
Use the FS as the primary source of information.  
Refer to the BS only if it helps clarify or expand on information in the FS.  
Do not use any external knowledge or fabricate any content.

---

FINAL SUMMARY (FS):
{final_summary}

---
BLOCK SUMMARY (BS):
{block_summary}

---
HIGHLIGHTED TEXT:
[{highlighted_text}]

---
USER REQUEST:
{user_prompt}

---
Respond by editing only the highlighted portion in [brackets] based on the user's request.  
Use the Block Summary only for clarification if needed.  
Keep the edit consistent with the overall style and tone of the Final Summary.
"""


In [55]:
def build_bs_all(block_summaries):
    """Build bs_all by combining all block summaries into a single text."""
    summaries = []
    
    for idx, block in enumerate(block_summaries):
        summary_text = block.get("summary", "").strip()
        if summary_text:  # Only add if it's non-empty
            summaries.append(f"{summary_text}")
    
    bs_all = "\n\n".join(summaries)
    return bs_all

In [56]:
def build_prompt_fs_plus_bs_all(final_summary: str, user_prompt: str) -> str:
    return f"""You are given two types of information:
- Final Summary (FS): A complete summary of the entire text.
- Block Summaries (BS_ALL): A collection of short summaries, each corresponding to different sections of the full text.

The user's request concerns the Final Summary or the full text.  
Use the FS as the main source of information.  
Use the Block Summaries (BS_ALL) to enrich, expand, or add missing details if needed.  
Do not use any external knowledge or fabricate any content.

Answer the user's request as accurately as possible, prioritizing information from the FS, and complementing it with Block Summaries where necessary.

---

FINAL SUMMARY (FS):
{final_summary}

---
BLOCK SUMMARIES (BS_ALL):
{build_bs_all(block_summaries)}

---
USER REQUEST:
{user_prompt}

---
Respond mainly based on the Final Summary.  
Use Block Summaries to add more depth, technical details, or examples if required.  
Do not invent new facts or introduce information not present in the FS or BS_ALL.
"""

In [57]:
def build_prompt_fs_plus_src_all(final_summary: str, user_prompt: str) -> str:
    """Builds the prompt when the task requires using Final Summary and all Source Text blocks."""
    
    # Combine all source blocks into one text
    src_all = "\n\n".join(source_blocks)

    return f"""You are given two types of information:
- Final Summary (FS): A complete summary of the full text.
- Source Text (SRC_ALL): The full detailed original text, divided into blocks.

The user's request concerns the Final Summary or the full text.  
Use the FS as the primary guide, but rely on the full Source Text (SRC_ALL) to add precision, mathematical proofs, citations, or details as needed.  
Do not use any external knowledge or fabricate content outside of the FS and SRC_ALL.

Answer the user's request as accurately as possible, using information from both FS and SRC_ALL.

---

FINAL SUMMARY (FS):
{final_summary}

---
SOURCE TEXT (SRC_ALL):
{src_all}

---
USER REQUEST:
{user_prompt}

---
Respond accurately based on both the Final Summary and the full Source Text provided.  
Do not invent new facts or introduce information not present in FS or SRC_ALL.
"""


In [58]:
def build_prompt_fs_plus_src(
    source_block: str, final_summary: str, user_prompt: str, highlighted_text: str = None
) -> str:
    """
    Builds the prompt when the task requires FS + a small snippet from Source Text (SRC[i]),
    optionally focusing on a highlighted section of the Final Summary.
    """

    if highlighted_text is None:
        # General request (global edit)
        return f"""You are given two pieces of information:
- Final Summary (FS): A complete summary of the entire text.
- Source Snippet (SRC): A small excerpt from the original Source Text, focused on a specific topic.

The user's request concerns the Final Summary or the full text.  
Use the FS as the main source of information.  
Use the Source Snippet (SRC) only if the user's request demands precise wording, quotes, numbers, formulas, or direct citations.  
Do not use any external knowledge or fabricate any content.

Answer the user's request as accurately as possible, prioritizing the FS, and using the SRC snippet only for exact references when necessary.

---

FINAL SUMMARY (FS):
{final_summary}

---
SOURCE SNIPPET (SRC):
{source_block}

---
USER REQUEST:
{user_prompt}

---
Respond mainly based on the Final Summary.  
Use the Source Snippet only if the user needs exact quotes, formulas, or original wording.  
Do not invent new facts or introduce information not present in FS or SRC.
"""
    else:
        # Scoped request (highlighted edit)
        return f"""You are given two pieces of information:
- Final Summary (FS): A complete summary of the entire text.
- Source Snippet (SRC): A small excerpt from the original Source Text, focused on a specific topic.

The user highlighted a specific portion of the Final Summary, enclosed in [brackets], and requested an edit for that part.  
Use the FS as the main source of information.  
Use the Source Snippet (SRC) only if the user's request demands precise wording, quotes, numbers, formulas, or direct citations.  
Do not use any external knowledge or fabricate any content.

---

FINAL SUMMARY (FS):
{final_summary}

---
SOURCE SNIPPET (SRC):
{source_block}

---
HIGHLIGHTED TEXT:
[{highlighted_text}]

---
USER REQUEST:
{user_prompt}

---
Respond by editing only the text inside the [brackets], based on the user's request.  
Use the Source Snippet only if the user needs exact references.  
Keep the edit consistent with the overall style and tone of the Final Summary.
"""


In [62]:
for item in labeled_prompts:
    user_prompt = item["prompt"]
    predicted_class = item["predicted_class"]

    generated_prompt = None
    print(predicted_class)

    if predicted_class == "FS_ONLY":
        generated_prompt = build_prompt_fs_only(current_summary, user_prompt)
    elif predicted_class == "FS_PLUS_BS[i]":
        matching_index = find_best_matching_index(block_summaries, user_prompt)
        matching_block = block_summaries[matching_index]
        matching_block_summary = matching_block["summary"]
        generated_prompt = build_prompt_fs_plus_bs(matching_block_summary, current_summary, user_prompt)
    elif predicted_class == "FS_PLUS_BS_ALL":
        generated_prompt = build_prompt_fs_plus_bs_all(current_summary, user_prompt)
    elif predicted_class == "FS_PLUS_SRC[i]":
        matching_index = find_best_matching_index(block_summaries, user_prompt)
        matching_text_block = source_blocks[matching_index]
        generated_prompt = build_prompt_fs_plus_src(matching_text_block, current_summary, user_prompt)
    elif predicted_class == "FS_PLUS_SRC":
        generated_prompt = build_prompt_fs_plus_src_all(current_summary, user_prompt)

    if generated_prompt is not None:
        response = openrouter_ask(generated_prompt, model="openai/gpt-4o-mini")
        print(f"User Prompt: {user_prompt}")
        print(response)


FS_ONLY
User Prompt: Summarize the key points of Nash equilibrium more briefly.
### 🎯 Nash Equilibrium
- A strategy profile σ* is a **Nash equilibrium** if no player can unilaterally deviate to improve their payoff: ui(σ*i, σ*-i) ≥ ui(si, σ*-i) for every player i and every pure strategy si ∈ Si.
- In a Nash equilibrium with mixed strategies, players must be indifferent among all pure strategies in the support of that mixed strategy.
- Strategies eliminated by iterated strict dominance cannot be played with positive probability in a Nash equilibrium.
- Nash equilibria are seen as "consistent predictions" or "stable conventions," assuming players have correct beliefs about opponents' strategies and act rationally.
FS_PLUS_BS[i]
User Prompt: Rewrite the section about dominated strategies in a simpler way.
### 📉 Dominated Strategies (Simplified)

A strategy is considered **strictly dominated** if there is another strategy that always gives a better payoff, no matter what the other players 

In [63]:
for item in scoped_labeled_prompts:
    user_prompt = item["user_prompt"]
    predicted_class = item["predicted_class"]
    highlighted_text = item["highlighted_text"]

    generated_prompt = None
    print(predicted_class)


    if predicted_class == "FS_ONLY":
        generated_prompt = build_prompt_fs_only(current_summary, user_prompt, highlighted_text)
    elif predicted_class == "FS_PLUS_BS[i]":
        matching_index = find_best_matching_index(block_summaries, user_prompt)
        matching_block = block_summaries[matching_index]
        matching_block_summary = matching_block["summary"]
        generated_prompt = build_prompt_fs_plus_bs(matching_block_summary, current_summary, user_prompt, highlighted_text)
    elif predicted_class == "FS_PLUS_SRC[i]":
        matching_index = find_best_matching_index(block_summaries, user_prompt)
        matching_text_block = source_blocks[matching_index]
        generated_prompt = build_prompt_fs_plus_src(matching_text_block, current_summary, user_prompt, highlighted_text)

    if generated_prompt is not None:
        response_scoped = openrouter_ask(generated_prompt, model="openai/gpt-4o-mini")
        print(f"User Prompt: {user_prompt}")
        print(f"Highlighted Text: {highlighted_text}")
        print(response_scoped)


FS_ONLY
User Prompt: Explain what 'strategic interaction' means.
Highlighted Text: Game Theory is the formal study of strategic interaction.
[Game Theory is the formal study of strategic interaction, which refers to situations where the outcome for each participant depends not only on their own actions but also on the actions of others. In these scenarios, players must consider the potential choices of their opponents and make decisions that optimize their own outcomes based on these considerations.]
FS_ONLY
User Prompt: Can you explain how players predict opponents' play?
Highlighted Text: Predicting opponents' play and responding optimally is key.
[Predicting opponents' play involves analyzing their past actions, understanding their strategies, and anticipating their responses based on the game's structure. Players use this information to formulate optimal responses that maximize their own payoffs.]
FS_PLUS_BS[i]
User Prompt: Expand on the recent applications mentioned here.
Highligh